In [ ]:
import numpy as np
%pylab inline
import matplotlib.pyplot as plt
from scipy import signal
import os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras import models, regularizers
from keras.utils.np_utils import to_categorical
import skimage.draw
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
def make_labels(directory, data=[], y_hat=[], label=0):
    for root, dirs, files in os.walk(directory):
        for file in files:
            img = matplotlib.image.imread(directory+file)
            data.append(img)
        y_hat = [label] * len(data)
    return np.array(data), np.array(y_hat)

In [ ]:
# https://www.kaggle.com/danbrice/keras-plot-history-full-report-and-grid-search
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


In [ ]:
circles, y_circles = [], []
circles, y_circles = make_labels('shapes/circles/', data=circles, y_hat=y_circles)

squares, y_squares = [], []
squares, y_squares = make_labels('shapes/squares/', data=squares, y_hat=y_squares, label=1)

triangles, y_triangles = [], []
triangles, y_triangles = make_labels('shapes/triangles/', data=triangles, y_hat=y_triangles, label=2)

In [ ]:
print(circles.shape, squares.shape, triangles.shape)
print(y_circles.shape, y_squares.shape, y_triangles.shape)

In [ ]:
X = np.vstack((circles, squares, triangles))
y = np.hstack((y_circles, y_squares, y_triangles)).reshape(-1, 1)
y_cat = to_categorical(y)

In [ ]:
i = 3
plt.imshow(circles[i])
plt.show()
plt.imshow(triangles[i])
plt.show()
plt.imshow(squares[i])
plt.show()

In [ ]:
img = triangles[3]
rows, cols, ch = img.shape 

### Rotation
Rotation around center `(cx,cy)` by angle `deg` degrees, and scale the image by `f`

`cv2.getRotationMatrix2D( (cx,cy), deg, f )`

In [ ]:
rot_mat = cv2.getRotationMatrix2D( (14,14), 70, 0.5 );

In [ ]:
rot_mat

In [ ]:
timg = cv2.warpAffine( img, rot_mat, (cols, rows), borderValue=(1.,1.,1.) )

In [ ]:
plt.imshow(timg)

### Affine transformation
Transform a triangle to an other triangle. `pts1` is transformed to `pts2`

In [ ]:
pts1 = np.float32([[5, 5], 
                   [20, 5],  
                   [5, 20]]) 
  
pts2 = np.float32([[1, 10], 
                   [20, 5],  
                   [10, 25]]) 
  
M = cv2.getAffineTransform(pts1, pts2) 
dst = cv2.warpAffine(img, M, (cols, rows), borderValue=(1.,1.,1.)) 

In [ ]:
plt.imshow(dst)

### Convolution

In [ ]:
kernel = np.array([[0,0.1,0],[0.1,0.6,0.1],[0,0.1,0]])
dst = np.clip(cv2.filter2D(img,-1,kernel),0,1)
plt.imshow(dst)

In [ ]:
plt.subplots(10,10,figsize=(15,15))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.imshow(triangles[i])

## TASK
Using the single training images above (`circles[3], squares[3], trinagles[3]`), create a few hundred new training images using the transformations above. Use it as a training set for a convolutional neural network and test it on the original data set. Accuracy above 40-50% is nice! Hints:
 * I used 4 layers of convolutions and max pooling after each second
 * The dense hidden layer can be large ~100 nodes but always use regulariaztion overfitting in this case is suicide!
 * I have used only affine transformation on the rotated image, but you can try other things
 * I have used a weak image enhancement with `[[0,-e,0],[-e,1+4e,-e],[0,-e,0]]` matrix, with `e` around 0.25
 * After all transformations, I have rescaled the images to the range $[0:1]$
 * I have used normally distributed random numbers on all possible parameters of the affine transformations, see example below for possible problems.
 * Advice: plot the created images (see the command above) to see if they get similarly distorted to the ones above

### Normally distributed integer numbers
General task: modify a number with an integer using normal distribution

Solution: generate normally distributed float numbers and convert it to integer. Problem is that both `int(0.5)=0` and `int(-0.5)=0`. Thus make sure that the float numbers are always positive. In the second example below, you can see the high peak at 0 which is artificial.

In [ ]:
plt.hist(np.random.normal(4.5,1.5,size=1000).astype(int),bins=np.arange(10)-0.5)

In [ ]:
plt.hist(np.random.normal(1.5,1.5,size=1000).astype(int),bins=np.arange(10)-4.5)

In [ ]:
int(-0.5)